# From Notebook to Kubeflow Pipeline using Fashion MNIST

In this notebook, we will walk you through the steps of converting a machine learning model, which you may already have on a jupyter notebook, into a Kubeflow pipeline. As an example, we will make use of the fashion we will make use of the fashion MNIST dataset and the [Basic classification with Tensorflow](https://www.tensorflow.org/tutorials/keras/classification) example.

In this example we use:

* **Kubeflow pipelines** - [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) is a machine learning workflow platform that is helping data scientists and ML engineers tackle experimentation and productionization of ML workloads. It allows users to easily orchestrate scalable workloads using an SDK right from the comfort of a Jupyter Notebook.

* **Microk8s** - [Microk8s](https://microk8s.io/docs) is a service that gives you the ability to spin up a lightweight Kubernetes cluster right on your local machine. It comes with Kubeflow built right in. 

**Note:** This notebook is to be run on a notebook server inside the Kubeflow environment. 

## Section 1: Data exploration (as in [here](https://www.tensorflow.org/tutorials/keras/classification))

The [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)  dataset contains 70,000 grayscale images in 10 clothing categories. Each image is 28x28 pixels in size. We chose this dataset to demonstrate the funtionality of Kubeflow Pipelines without introducing too much complexity in the implementation of the ML model.

To familiarize you with the dataset we will do a short exploration. It is always a good idea to understand your data before you begin any kind of analysis.

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>


### 1.1 Install packages:

In [ ]:
!python -m pip install --user --upgrade pip
!pip install --user --upgrade pandas matplotlib numpy

After the installation, we need to restart kernel for changes to take effect:

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### 1.2 Import libraries

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Data exploration libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 1.2 Import the Fashion MNIST dataset

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Each image is mapped to a single label. Since the *class names* are not included with the dataset, store them here to use later when plotting the images:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Let's look at the format of each dataset split. The training set contains 60,000 images and the test set contains 10,000 images which are each 28x28 pixels.

---



### 1.3 Explore the data

In [ ]:
print(f'Number of training images: {train_images.shape[0]}\n')
print(f'Number of test images: {test_images.shape[0]}\n')

print(f'Image size: {train_images.shape[1:]}')


There are logically 60,000 training labels and 10,000 test labels.

In [ ]:
print(f'Number of labels: {len(train_labels)}\n')
print(f'Number of test labels: {len(test_labels)}')

Each label is an integer between 0 and 9 corresponding to the 10 class names.

In [ ]:
unique_train_labels = np.unique(train_labels)

for label in zip(class_names, train_labels):
  label_name, label_num = label
  print(f'{label_name}: {label_num}')

### 1.4 Preprocess the data

To properly train the model, the data must be normalized so each value will fall between 0 and 1. Later on this step will be done inside of the training script but we will show what that process looks like here.

The first image shows that the values fall in a range between 0 and 255.

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

To scale the data we divide the training and test values by 255.

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

We plot the first 25 images from the training set to show that the data is in fact in the form we expect.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()